In [45]:
import re, torch
import torch.nn as nn
torch.manual_seed(42)

말뭉치 `docs`와 타깃 레이블 `label`을 수동으로 만듭니다. 전체 클래스 개수는 3개입니다.

In [48]:
docs = [
    "Movies are fun for everyone.",
    "Watching movies is great fun.",
    "Enjoy a great movie today.",
    "Research is interesting and important.",
    "Learning math is very important.",
    "Science discovery is interesting.",
    "Rock is great to listen to.",
    "Listen to music for fun.",
    "Music is fun for everyone.",
    "Listen to folk music!"
]

labels = [1, 1, 1, 3, 3, 3, 2, 2, 2, 2]
num_classes = len(set(labels))

num_classes

3

파이썬 정규식을 사용해 텍스트를 단순하게 공백을 기준으로 단어로 분리합니다.

In [50]:
def tokenize(text):
    '''
    대문자가 섞일 경우 별도의 토큰으로 취급되지 않게하기 위해 모든 텍스트를 소문자화
    '''
    return re.findall(r"\w+", text.lower())

In [51]:
tokenize(docs[0])

['movies', 'are', 'fun', 'for', 'everyone']

`texts`를 순회하면서 각 텍스트에서 모든 단어(토큰)을 추출합니다. 고유한 단어 모음을 만들기 위해 파이썬의 집합 컴프리헨션을 사용합니다.
그 다음 집합의 원소를 알파벳 순으로 정렬하여 인덱스를 부여하는 작업을 딕셔너리 컴프리헨션으로 수행합니다. 결과적으로 `단어:인덱스`의 딕셔너리가 생성됩니다.

In [53]:
def get_vocabulary(texts):
    '''
    docs의 로우별로 추출된 토큰들(array)를 순회하며 중복제거된 토큰 목록을 만들고 알파벳 순으로 정렬하여 idx 부여한 결과를 return
    말뭉치로부터 어휘 사전을 만든다.
    '''
    tokens = {token for text in texts for token in tokenize(text)}
    return {word: idx for idx, word in enumerate(sorted(tokens))}

In [54]:
vocabulary = get_vocabulary(docs)
vocabulary

{'a': 0,
 'and': 1,
 'are': 2,
 'discovery': 3,
 'enjoy': 4,
 'everyone': 5,
 'folk': 6,
 'for': 7,
 'fun': 8,
 'great': 9,
 'important': 10,
 'interesting': 11,
 'is': 12,
 'learning': 13,
 'listen': 14,
 'math': 15,
 'movie': 16,
 'movies': 17,
 'music': 18,
 'research': 19,
 'rock': 20,
 'science': 21,
 'to': 22,
 'today': 23,
 'very': 24,
 'watching': 25}

텍스트를 입력 받으면 어휘사전 길이의 벡터를 생성하고 어휘사전에 등장하는 단어 위치를 1로 설정하는 함수를 만듭니다.
- bow (Bag of Words) : 각 단어의 출현 빈도를 표시하는 문법론

In [55]:
def doc_to_bow(doc, vocabulary):
    '''
    입력 텍스트가 어휘 사전에 존재하는지 확인하고 존재하면 1로 설정
    '''
    tokens = set(tokenize(doc))
    bow = [0] * len(vocabulary)
    for token in tokens:
        if token in vocabulary:
            bow[vocabulary[token]] = 1
    return bow

말뭉치에 있는 모든 텍스트를 `doc_to_bow` 함수를 사용해 문서 단어 행렬로 만듭니다. 레이블을 텐서로 변환합니다.

In [56]:
vectors = torch.tensor(
    [doc_to_bow(doc, vocabulary) for doc in docs],
    dtype=torch.float32
)
# 머신러닝 라이브러리에서는 클래스 레이블이 0부터 시작하는게 일반적이기 떄문에 -1
labels = torch.tensor(labels, dtype=torch.long) - 1

`vectors`는 문서 단어 행렬입니다.

In [57]:
vectors

tensor([[0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0.,

신경망의 입력 크기는 어휘사전의 크기에 해당합니다.
- 어휘사전이 커지면 커질수록 입력차원이 커지기 때문에 최신 신경망에서는 bow가 아닌 단어 벡터를 대신하여 사용

In [58]:
# 입력 차원은 어휘사전 길이
input_dim = len(vocabulary)
hidden_dim = 50 # 은닉 차원
output_dim = num_classes

input_dim

26

다중 분류에서 마지막 층의 활성화 함수는 소프트맥스 함수를 사용합니다.

$\text{softmax}(\mathbf{z}, k) = \dfrac{e^{z_k}}{\displaystyle \sum_{j=1}^D e^{z_j}}$

파이토치에서 신경망에 소프트맥스 활성화 함수를 사용하는 경우 `NLLLoss` 손실 함수를 사용하고, 활성화 함수를 생략하는 경우 `CrossEntropyLoss`를 사용합니다.

In [59]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = SimpleClassifier(input_dim, hidden_dim, output_dim)

크로스 엔트로피 함수를 로지스틱 손실의 다중 분류 버전으로 생각할 수 있습니다.

$\text{loss}(\mathbf{\tilde{y}}, \mathbf{y}) = -\displaystyle \sum_{k=1}^{C} y^{(k)}\text{log}(\tilde{y}^{(k)})$

In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for step in range(3000):
    optimizer.zero_grad()
    loss = criterion(model(vectors), labels)
    loss.backward()
    optimizer.step()

In [61]:
new_docs = [
    "Listening to rock music is fun.",
    "I love science very much"
]

class_names = ['Cinema', 'Music', 'Science']

In [62]:
new_doc_vectors = torch.tensor(
    [doc_to_bow(new_doc, vocabulary) for new_doc in new_docs],
    dtype=torch.float32
)

with torch.no_grad():
    outputs = model(new_doc_vectors)
    predicted_ids = torch.argmax(outputs, dim=1) + 1

for i, new_doc in enumerate(new_docs):
    print(f'{new_doc}: {class_names[predicted_ids[i].item() - 1]}')


Listening to rock music is fun.: Music
I love science very much: Science
